In [1]:
!pip install -U scikit-learn

     |████████████████████████████████| 6.8MB 7.5MB/s eta 0:00:01
     |████████████████████████████████| 25.9MB 39.1MB/s eta 0:00:01�█████▎             | 14.8MB 39.1MB/s eta 0:00:01MB 39.1MB/s eta 0:00:01
     |████████████████████████████████| 307kB 40.5MB/s eta 0:00:01
     |████████████████████████████████| 14.5MB 19.5MB/s eta 0:00:01�███████████████▌            | 8.8MB 19.5MB/s eta 0:00:01     |███████████████████████████████▍| 14.2MB 19.5MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pandas

     |████████████████████████████████| 9.5MB 12.1MB/s eta 0:00:01
     |████████████████████████████████| 512kB 34.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install lightgbm

     |████████████████████████████████| 1.8MB 10.1MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
def predict(test_data, model_name, clf):
    from sklearn import metrics
    y_pred=clf.predict(test_data)
    print(f"prediction: {y_pred}")
    # AUC
    print(f"auc: {metrics.roc_auc_score(y_test, y_pred)}")
    # convert probabilities to binary prediction using threshold=0.5
    for i in range(0,y_pred.shape[0]):
        if y_pred[i]>=.5: # setting threshold to .5
            y_pred[i]=1
        else:
            y_pred[i]=0
    # Confusion matrix
    cm = metrics.confusion_matrix(y_test, y_pred)
    print(f"Confusion matrix:\n {cm}")
    # Accuracy
    accuracy=metrics.accuracy_score(y_pred,y_test)
    print(f"Accuracy: {accuracy}")
    # Precision
    precision = metrics.precision_score(y_pred,y_test, average='micro')
    print(f"Precision: {precision}")
    # Recall
    recall = metrics.recall_score(y_pred,y_test, average='micro')
    print(f"Recall: {recall}")

In [7]:
#Adult Census Income Binary classfication dataset
df = pd.read_csv('https://raw.githubusercontent.com/habtamu/demo/main/dataset.csv')
new_df = df.loc[:, ~df.columns.isin([' workclass',' occupation',' native-country'])]
le = LabelEncoder()
new_df.loc[:,' education'] = le.fit_transform(new_df.loc[:,' education'].values.tolist())
new_df.loc[:,' marital-status'] = le.fit_transform(new_df.loc[:,' marital-status'].values.tolist())
new_df.loc[:,' relationship'] = le.fit_transform(new_df.loc[:,' relationship'].values.tolist())
new_df.loc[:,' race'] = le.fit_transform(new_df.loc[:,' race'].values.tolist())
new_df.loc[:,' sex'] = le.fit_transform(new_df.loc[:,' sex'].values.tolist())
new_df.loc[:,' income'] = le.fit_transform(new_df.loc[:,' income'].values.tolist())
X = new_df.drop(' income', axis=1)
y = new_df[' income']

#use stratify for un balanced number of examples for each class label
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.3,
                                                        #stratify=y,
                                                        random_state=0)

# Experiment 1: using Boosted Decision Tree [LightGBM]
import lightgbm as lgb
MODEL = 'LightGBM'
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.2
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'auc'
params['num_leaves'] = 20
params['min_data'] = 10
#params['max_depth'] = 10
#params['sub_feature'] = 0.5

# data is ready to train
clf = lgb.train(params, d_train, 100)
# predict
print(f"----------{MODEL}--------------")
predict(X_test, model_name=MODEL, clf=clf)
print(f"-----------END--------------")

# Experiment 2: using Support Vector Machines
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
MODEL = 'SVM'
clf = make_pipeline(StandardScaler(), 
                    LinearSVC(penalty='l1', C=0.001, dual=False, max_iter=1000, random_state=42))
#clf = make_pipeline(StandardScaler(), SVC(C=0.001, kernel='linear', random_state=42))
clf.fit(X_train, y_train)
print(clf)
    
# predict
print(f"----------{MODEL}--------------")
predict(X_test, model_name=MODEL, clf=clf)
print(f"-----------END--------------")


[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5479, number of negative: 17313
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001298 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 618
[LightGBM] [Info] Number of data points in the train set: 22792, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240391 -> initscore=-1.150535
[LightGBM] [Info] Start training from score -1.150535
----------LightGBM--------------
prediction: [0.02165759 0.02283388 0.1498168  ... 0.9988587  0.42303509 0.00596977]
auc: 0.919577699974176
Confusion matrix:
 [[6879  528]
 [ 827 1535]]
Accuracy: 0.8612959361244754
Precision: 0.8612959361244754
Recall: 0.8612959361244754
-----------END--------------
Pipeline(steps=[('standardscaler', StandardScaler()),
      